<a href="https://colab.research.google.com/github/D34dP0oL/4216_Biomedical_DS_and_AI/blob/main/Sheet7/Assignment7_Solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import math
import pandas as pd
import random as rand

In [ ]:
def get_dataset_from_github(filename, index_col_str=None, header_str='infer'):    
    data_file_path = "https://raw.githubusercontent.com/D34dP0oL/4216_Biomedical_DS_and_AI/main/Datasets/"
    if index_col_str is None and header_str == 'infer':
      data = pd.read_csv(data_file_path + filename)
    elif index_col_str is None:
        data = pd.read_csv(data_file_path + filename, header=header_str)
    elif header_str == 'infer':
      data = pd.read_csv(data_file_path + filename, index_col=index_col_str)
    else:
      data = pd.read_csv(data_file_path + filename, index_col=index_col_str, header=header_str)

    return data

## Biomedical Data Science & AI

## Assignment 7

#### Group members:  Fabrice Beaumont, Fatemeh Salehi, Genivika Mann, Helia Salimi, Jonah

---
### Exercise 1 - Elastic Net & Nested Cross-Validation


#### 1.1. Using the `titanic_survival_data.csv` dataset, train a logistic regression model with elastic net penalization to demonstrate the pros and cons of the different data splitting methods and give a short description on what you observe.

##### 1.1.a) Report the accuracy of data splitting with a test size of $0.2$ and random state as $1$.

##### 1.1.b) Plot the boxplot for the accuracy of the **$K$-fold cross validation** with $5$ splits.

##### 1.1.c) Plot the boxplot for the accuracy of the **Stratified-$K$-fold cross validation** with $5$ splits.

##### 1.1.d) Inform yourself about **leave-one-out cross-validation** (**LOOCV**). Implement LOOCV and mention the pros and cons of the method.

#### 1.2. Use the nested cross validation to train a logistic regression with elastic net penalization (`leukemia_small.csv`).

##### 1.2.a) Split the data into training and test samples using an appropriate cross validation method, and in the inner loop carry out **hyperparameter optimization**.

##### 1.2.b) Compute the area under the ROC curve (**AUC-ROC**) and the area under the precision-recall curve (**AUC-PR**).

##### 1.2.c) Plot separate boxplots for the two performance metrics.

#### 1.3. In your own words, explain how each of the following metrics can be used to assess the performance of a model and then calculate each metric using the following confusion matrix.

 _           | Predicted No | Predicted Yes |
---|---|---
Actual No    | $250$        | $20$          |
Actual Yes   | $30$         | $100$         |

##### 1.3.a) Recall

##### 1.3.b) $F_1$

##### 1.3.c) Balanced Accuracy (BAC)

##### 1.3.d) Matthews Correlation Coefficient (MCC)

---
### Exercise 2 - SVM

#### 2.1. Inform yourself about **SVM** and briefly explain the working strategy of linear SVM and why maximizing the margin is a good strategy.

#### 2.2. Inform yourself about the non-linearity problem for classifiers. Briefly explain how SVM uses **kernel trick** to overcome this issue.

---
### Exercise 3 - Random Forest

For the following questions, use `random_seed = 1` for better reproducibility of your
answers.

#### 3.1. Load the breast cancer dataset from sklearn to your Jupyter notebook. Use label encoding to convert your target variable “class” into numerical form. Split the dataset using a $5$-fold cross validation.

#### 3.2. Set up a parameter grid and use grid search with $5$-fold cross validation to identify the best hyperparameter values used to fit a random forest classifier.

#### 3.3. Use the best hyperparameters from *2)* to fit the final model. Predict the classes of the test set and count the number of samples assigned to each class.

#### 3.4. Print the importance of each feature in descending order. Identify the top five features.

#### 3.5. Mention a case when permutation feature importance is favored over impurity-based feature importance. Use permutation importance to print the importances of your features in a descending order. Compare your answer with that of *4)*. Do you notice any differences?

#### 3.6. In your own words, explain the **bootstrapping technique** and mention how random forest benefits from its application.